In [ ]:
# importing essential libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix, classification_report 

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras import layers

In [ ]:
path = r"C:\Users\Amatari Ebi\Desktop\Data Science\TechCrush\Artificial Intelligence\Face Mask dataset"

In [ ]:
# specifying image dimension and batch size

img_size = (224, 224)
batchSize = 32

In [ ]:
# defining instance
datagen = ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

In [ ]:
# creating train set
train_set = datagen.flow_from_directory(
    path,
    target_size=img_size,
    batch_size=batchSize,
    class_mode="categorical",
    subset="training",
    shuffle=True,
)

In [ ]:
# creating test set
test_set = datagen.flow_from_directory(
    path,
    target_size=img_size,
    batch_size=batchSize,
    class_mode="categorical",
    subset="validation",
    shuffle=False
)

In [ ]:
# checking the values of labels
print(train_set.class_indices)

In [ ]:
model = keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(train_set.num_classes, activation='softmax')
])

In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["Accuracy"])

In [ ]:
history = model.fit(train_set, epochs=10, validation_data=test_set) 

In [ ]:
test_loss, test_acc = model.evaluate(test_set)
print(f"Accuracy: {test_acc:.4f}")

In [ ]:
base_model = keras.applications.EfficientNetB0(
    input_shape=(224,224,3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False

In [ ]:
model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')
])

In [ ]:
model.compile(
    optimizer="adam",
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(train_set, epochs=5, validation_data=test_set) 

In [ ]:
test_loss, test_acc = model.evaluate(test_set)
print(f"Accuracy: {test_acc:.4f}")

In [ ]:

y_true = test_set.classes

y_pred = model.predict(test_set)
y_pred = np.argmax(y_pred, axis=1)

print(classification_report(y_true, y_pred))

In [ ]:

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(9,6))
sns.heatmap(cm, annot=True, cmap="Blues")
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.show()

In [ ]:
#Extracts a batch of test_set images
images, labels = next(test_set)

In [ ]:
predictions = model.predict(images)

In [ ]:
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(labels, axis=1)

In [ ]:
class_names = list(train_set.class_indices.keys())
print(class_names)

In [ ]:
plt.figure(figsize=(12, 8))

for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(images[i])
    plt.axis("off")

    true_label = class_names[true_classes[i]]
    predicted_label = class_names[predicted_classes[i]]

    title_color = "green" if true_label == predicted_label else "red"
    plt.title(f"True: {true_label}\nPred: {predicted_label}", color=title_color)

plt.tight_layout()
plt.show()

In [ ]:
train_set.classes